This notebook covers the basic of creating UDTFs for manipulating a `GeoPoint`. A `GeoPoint` is a struct holding two coordinates: `x` and `y`.

```c
struct Point2D {
    float x;
    float y;
};
```

For the API, check the RBC ReadTheDocs page:
* [`Column<GeoPoint>`](https://rbc.readthedocs.io/en/latest/generated/rbc.heavydb.ColumnGeoPoint.html#rbc.heavydb.ColumnGeoPoint)
* [`GeoPoint`](https://rbc.readthedocs.io/en/latest/generated/rbc.heavydb.GeoPoint.html#rbc.heavydb.GeoPoint)

In [1]:
import warnings; warnings.filterwarnings('ignore')

### Connect to the HeavyDB server

In [2]:
# NBVAL_IGNORE_OUTPUT
from rbc.heavydb import RemoteHeavyDB
heavydb = RemoteHeavyDB(user='admin', password='HyperInteractive',
                        host='127.0.0.1', port=6274)

GeoPoint requires HeavyDB 7.0 or newer

In [3]:
heavydb.version[:3]

(7, 0, 0)

### Load test data

In [4]:
from util import load_test_data
from rbc.tests import _PointTestTable
load_test_data(heavydb, _PointTestTable, 'point_table')

List of points in "point_table"

In [5]:
import pandas as pd
descr, points = heavydb.sql_execute('select * from point_table')
pd.DataFrame(list(points), columns=map(lambda x: x.name, descr))

,p1,p2,p3,p4
0,POINT (1 2),POINT (2.99999999022111 3.99999997299165),POINT (5 6),POINT (7 8)
1,POINT (9 8),POINT (6.99999992130324 5.99999998044223),POINT (5 4),POINT (3 2)
2,None,None,None,None


### Define a function that operate on GeoPoints

`rbc_ct_shift` shifts each point by a value defined in the SQL query

In [6]:
from rbc.heavydb import Point2D

@heavydb("int32(TableFunctionManager, Column<T>, double, double, OutputColumn<T>)",
         T=['GeoPoint'], devices=['cpu'])
def rbc_ct_shift(mgr, points, x, y, shifted_points):
    size = len(points)
    mgr.set_output_row_size(size)
    for i in range(size):
        if points.is_null(i):
            shifted_points.set_null(i)
        else:
            point = points[i]
            shifted_points.set_item(i, Point2D(point.x + x, point.y + y))
    return size

In [7]:
col = 'p1'
table_name = 'point_table'

Original set of points in `p1` column:

In [8]:
descr, result = heavydb.sql_execute(f'select p1 from {table_name}')
pd.DataFrame(list(result), columns=map(lambda x: x.name, descr))

,p1
0,POINT (1 2)
1,POINT (9 8)
2,None


Shift each point `p` by:
* `p.x += 2`
* `p.y += 10`

In [9]:
shift_x = '2'
shift_y = '10'

query = (f'SELECT * FROM TABLE(rbc_ct_shift('
         f'cursor(SELECT {col} FROM {table_name}), '
         f'{shift_x}, {shift_y}));')

descr, result = heavydb.sql_execute(query)
pd.DataFrame(list(result), columns=map(lambda x: x.name, descr))

'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)
'znver3' is not a recognized processor for this target (ignoring processor)


,out0
0,POINT (3 12)
1,POINT (11 18)
2,None


Other examples can be found in the rbc test suite for `GeoPoint`:

https://github.com/xnd-project/rbc/blob/main/rbc/tests/heavydb/test_geopoint.py